## Imports

In [2]:
import pandas as pd
import numpy as np
import os

## PD Set Options

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1600)

## Read in and Analysis

In [4]:
df = pd.read_csv('Property_Point_View.csv', dtype={'FOLIO':'str'}, low_memory=False)

In [5]:
df.columns

Index(['X', 'Y', 'OBJECTID', 'FOLIO', 'TTRRSS', 'X_COORD', 'Y_COORD',
       'TRUE_SITE_ADDR', 'TRUE_SITE_UNIT', 'TRUE_SITE_CITY',
       'TRUE_SITE_ZIP_CODE', 'TRUE_MAILING_ADDR1', 'TRUE_MAILING_ADDR2',
       'TRUE_MAILING_ADDR3', 'TRUE_MAILING_CITY', 'TRUE_MAILING_STATE',
       'TRUE_MAILING_ZIP_CODE', 'TRUE_MAILING_COUNTRY', 'TRUE_OWNER1',
       'TRUE_OWNER2', 'TRUE_OWNER3', 'CONDO_FLAG', 'PARENT_FOLIO',
       'DOR_CODE_CUR', 'DOR_DESC', 'SUBDIVISION', 'BEDROOM_COUNT',
       'BATHROOM_COUNT', 'HALF_BATHROOM_COUNT', 'FLOOR_COUNT', 'UNIT_COUNT',
       'BUILDING_ACTUAL_AREA', 'BUILDING_HEATED_AREA', 'LOT_SIZE',
       'YEAR_BUILT', 'ASSESSMENT_YEAR_CUR', 'ASSESSED_VAL_CUR', 'DOS_1',
       'PRICE_1', 'LEGAL', 'PID', 'DATEOFSALE_UTC'],
      dtype='object')

In [6]:
CONDITIONS = [
    df['DOR_DESC'].str.contains('RESIDENTIAL') == True,
    df['DOR_DESC'].str.contains('COMMERCIAL') == True,
    df['DOR_DESC'].str.contains('RELIGIOUS') == True
]

OUTPUTS = [
    'RESIDENTIAL',
    'COMMERCIAL',
    'RELIGIOUS'
]

res = np.select(CONDITIONS, OUTPUTS, 'Other')
res

df['ZONE'] = pd.Series(res)

In [ ]:
df.to_csv('Miami_Dade_Zones.csv')

## Create Religious Specific Pins

In [ ]:
df_RELIGIOUS = df.loc[df['DOR_DESC'].str.contains('RELIGIOUS',na=False)]

In [ ]:
print(len(df))
print('--------')
print(len(df_RELIGIOUS))

In [ ]:
df_RELIGIOUS = df_RELIGIOUS.fillna(value='None')

In [ ]:
# df.agg({'LOT_SIZE':['sum','min','max','mean','median']}).reset_index()

In [ ]:
# df_RELIGIOUS.agg({'LOT_SIZE':['sum','min','max','mean','median']}).reset_index()

In [ ]:
# df.agg({'BUILDING_ACTUAL_AREA':['sum','min','max','mean','median']}).reset_index()

In [ ]:
# df_RELIGIOUS.agg({'BUILDING_ACTUAL_AREA':['sum','min','max','mean','median']}).reset_index()

In [ ]:
def popup_html(row):
    i = row
    folio = df_RELIGIOUS['FOLIO'].iloc[i]
    address = df_RELIGIOUS['TRUE_SITE_ADDR'].iloc[i]
    year_built = df_RELIGIOUS['YEAR_BUILT'].iloc[i]
    building_area = df_RELIGIOUS['BUILDING_ACTUAL_AREA'].iloc[i]
    lot_size = df_RELIGIOUS['LOT_SIZE'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Folio: <strong>{}'''.format(folio) + '''</strong><br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Year Built: <strong>{}'''.format(year_built) + '''</strong><br>
    Building Area: <strong>{}'''.format(building_area) + '''</strong><br>
    Lot Size: <strong>{}'''.format(lot_size) + '''</strong><br>
    </html>
    '''
    return html


In [ ]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df_RELIGIOUS[["Y", "X"]].mean().to_list(),zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Miami-Dade Religious-Zoned Properties')

for i in range(0,len(df_RELIGIOUS)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df_RELIGIOUS['Y'].iloc[i],df_RELIGIOUS['X'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df_RELIGIOUS[['Y','X']].mean().to_list(),max_zoom=15)
m

In [ ]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [ ]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [ ]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)`